<a href="https://colab.research.google.com/github/salimzzz/ml-learning-projects/blob/main/autopayroll.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

employees = pd.read_csv("employees.csv")
print("Employees loaded:", len(employees))
employees.head()

Employees loaded: 50


,employee_id,name,base_salary,allowed_paid_leaves
0,E001,Rahul Kumar,25000,2
1,E002,Aisha Khan,30000,2
2,E003,Arjun Mehta,27000,2
3,E004,Priya Singh,26000,2
4,E005,Anjali Verma,28000,2


In [ ]:
import pandas as pd
import numpy as np

# Load employee data
employees = pd.read_csv("employees.csv")

# Dates for 26 working days in December 2025
dates = pd.date_range(start="2025-12-01", end="2025-12-26", freq="D")

records = []

# Fix random seed for consistent results
np.random.seed(10)

# More Present, few leaves
status_choices = ["Present", "Present", "Present", "Leave_Paid", "Leave_Unpaid"]

for emp_id in employees["employee_id"]:
    for d in dates:
        status = np.random.choice(status_choices, p=[0.70,0.10,0.05,0.10,0.05])
        records.append({
            "date": d.strftime("%Y-%m-%d"),
            "employee_id": emp_id,
            "status": status
        })

attendance = pd.DataFrame(records)

# Save to CSV
attendance.to_csv("attendance.csv", index=False)

print("attendance.csv created successfully with", len(attendance), "rows")
attendance.head()


attendance.csv created successfully with 1300 rows


,date,employee_id,status
0,2025-12-01,E001,Present
1,2025-12-02,E001,Present
2,2025-12-03,E001,Present
3,2025-12-04,E001,Present
4,2025-12-05,E001,Present


In [ ]:
import pandas as pd
from datetime import datetime

# Load employees and attendance
employees = pd.read_csv("employees.csv")
attendance = pd.read_csv("attendance.csv")

print("Employees:", len(employees))
print("Attendance rows:", len(attendance))

# Try to load existing salary_payments, else create empty
try:
    salary_payments = pd.read_csv("salary_payments.csv")
    print("Loaded existing salary_payments:", len(salary_payments), "rows")
except FileNotFoundError:
    salary_payments = pd.DataFrame(columns=[
        'employee_id', 'month', 'year',
        'gross_salary', 'total_deductions', 'net_salary',
        'paid', 'paid_date'
    ])
    print("No existing salary_payments found. Created empty DataFrame.")


Employees: 50
Attendance rows: 1300
Loaded existing salary_payments: 50 rows


In [ ]:
def calculate_salary_for_employee(emp_id, month, year, total_working_days=26):
    """
    Calculate salary for a single employee for a given month & year.
    Rules:
    - base_salary: full month salary
    - per_day_salary = base_salary / total_working_days
    - Leave_Paid: up to allowed_paid_leaves = no deduction
    - Extra paid leaves + Leave_Unpaid = unpaid, deducted
    """

    # Get employee details
    emp_row = employees[employees['employee_id'] == emp_id]
    if emp_row.empty:
        print(f"[WARN] No employee found with ID {emp_id}")
        return None

    base_salary = float(emp_row['base_salary'].iloc[0])
    allowed_paid_leaves = int(emp_row['allowed_paid_leaves'].iloc[0])

    # Ensure attendance date is datetime
    attendance_local = attendance.copy()
    attendance_local['date'] = pd.to_datetime(attendance_local['date'])

    # Filter attendance for this employee + month + year
    mask = (
        (attendance_local['employee_id'] == emp_id) &
        (attendance_local['date'].dt.month == month) &
        (attendance_local['date'].dt.year == year)
    )
    att_emp = attendance_local[mask]

    if att_emp.empty:
        # No attendance = assume full pay (you can change logic if you want)
        gross_salary = base_salary
        total_deductions = 0
        net_salary = gross_salary
    else:
        present_days = (att_emp['status'] == 'Present').sum()
        paid_leave_days_actual = (att_emp['status'] == 'Leave_Paid').sum()
        unpaid_leave_days = (att_emp['status'] == 'Leave_Unpaid').sum()

        # Paid leaves allowed without deduction
        paid_leave_days = min(paid_leave_days_actual, allowed_paid_leaves)

        # Extra paid leaves beyond allowed become unpaid
        extra_paid_leaves = max(0, paid_leave_days_actual - allowed_paid_leaves)
        unpaid_leave_days += extra_paid_leaves

        per_day_salary = base_salary / total_working_days

        total_deductions = unpaid_leave_days * per_day_salary
        gross_salary = base_salary
        net_salary = gross_salary - total_deductions

    result = {
        'employee_id': emp_id,
        'month': month,
        'year': year,
        'gross_salary': round(gross_salary, 2),
        'total_deductions': round(total_deductions, 2),
        'net_salary': round(net_salary, 2),
        'paid': False,
        'paid_date': ""
    }

    return result


def calculate_salary_for_all(month, year, total_working_days=26):
    """
    Calculate salary for ALL employees for a given month & year.
    Overwrites any old records for that same month & year,
    then appends fresh ones.
    """

    global salary_payments

    records = []
    for emp_id in employees['employee_id']:
        rec = calculate_salary_for_employee(emp_id, month, year, total_working_days)
        if rec is not None:
            records.append(rec)

    df_new = pd.DataFrame(records)

    # Remove existing rows for same month/year (if any)
    salary_payments = salary_payments[
        ~((salary_payments['month'] == month) &
          (salary_payments['year'] == year))
    ]

    # Append new calculations
    salary_payments = pd.concat([salary_payments, df_new], ignore_index=True)

    print(f"Calculated salaries for {len(df_new)} employees for {month}/{year}")
    return salary_payments


In [ ]:
# Example: calculate salary for December 2025
salary_payments = calculate_salary_for_all(month=12, year=2025, total_working_days=26)

# Show first 10 rows
salary_payments.head(10)


Calculated salaries for 50 employees for 12/2025


,employee_id,month,year,gross_salary,total_deductions,net_salary,paid,paid_date
0,E001,12,2025,25000.0,961.54,24038.46,False,
1,E002,12,2025,30000.0,1153.85,28846.15,False,
2,E003,12,2025,27000.0,2076.92,24923.08,False,
3,E004,12,2025,26000.0,2000.00,24000.00,False,
4,E005,12,2025,28000.0,0.00,28000.00,False,
5,E006,12,2025,24000.0,1846.15,22153.85,False,
6,E007,12,2025,22000.0,0.00,22000.00,False,
7,E008,12,2025,23000.0,0.00,23000.00,False,
8,E009,12,2025,29000.0,3346.15,25653.85,False,
9,E010,12,2025,31000.0,1192.31,29807.69,False,


In [ ]:
def get_paid_status(month, year):
    global salary_payments

    # Filter for the specific month and year
    filtered_salaries = salary_payments[
        (salary_payments['month'] == month) &
        (salary_payments['year'] == year)
    ]

    paid_df = filtered_salaries[filtered_salaries['paid'] == True]
    pending_df = filtered_salaries[filtered_salaries['paid'] == False]

    print(f"\n=== PAID Employees for {month}/{year} ===")
    if not paid_df.empty:
        print(paid_df[['employee_id', 'net_salary', 'paid_date']].to_string(index=False))
    else:
        print("No salaries paid yet.")

    print(f"\n=== PENDING Employees for {month}/{year} ===")
    if not pending_df.empty:
        print(pending_df[['employee_id', 'net_salary']].to_string(index=False))
    else:
        print("All salaries paid.")

def mark_salary_as_paid(emp_id, month, year):
    global salary_payments

    mask = (
        (salary_payments['employee_id'] == emp_id) &
        (salary_payments['month'] == month) &
        (salary_payments['year'] == year)
    )

    if not mask.any():
        print(f"No salary record found for {emp_id} in {month}/{year}")
        return

    # Update paid status
    salary_payments.loc[mask, 'paid'] = True
    salary_payments.loc[mask, 'paid_date'] = datetime.today().strftime('%Y-%m-%d')

    # Auto save
    salary_payments.to_csv("salary_payments.csv", index=False)

    print(f"Marked salary as PAID for {emp_id} for {month}/{year}")

    # Auto show updated report
    get_paid_status(month, year)

    # The line below caused the error and needs to be adjusted based on the new get_paid_status
    # It's better to show the full report rather than only pending employees in this context
    # If the user still wants to see a filtered view, we can add it here explicitly after get_paid_status


# CALL the function so it actually shows output
get_paid_status(12, 2025)


=== PAID Employees for 12/2025 ===
No salaries paid yet.

=== PENDING Employees for 12/2025 ===
employee_id  net_salary
       E001    24038.46
       E002    28846.15
       E003    24923.08
       E004    24000.00
       E005    28000.00
       E006    22153.85
       E007    22000.00
       E008    23000.00
       E009    25653.85
       E010    29807.69
       E011    20192.31
       E012    25961.54
       E013    23000.00
       E014    21153.85
       E015    28000.00
       E016    25000.00
       E017    26769.23
       E018    27692.31
       E019    21230.77
       E020    25961.54
       E021    25000.00
       E022    22115.38
       E023    28000.00
       E024    28615.38
       E025    25653.85
       E026    22115.38
       E027    21153.85
       E028    24000.00
       E029    30000.00
       E030    23884.62
       E031    23692.31
       E032    23000.00
       E033    22153.85
       E034    24538.46
       E035    28846.15
       E036    26230.77
       E037    

In [ ]:
from datetime import datetime

def mark_salary_as_paid(emp_id, month, year):
    global salary_payments, employees

    mask = (
        (salary_payments['employee_id'] == emp_id) &
        (salary_payments['month'] == month) &
        (salary_payments['year'] == year)
    )

    if not mask.any():
        print(f"No salary record found for {emp_id} in {month}/{year}")
        return

    # Update fields
    salary_payments.loc[mask, 'paid'] = True
    salary_payments.loc[mask, 'paid_date'] = datetime.today().strftime('%Y-%m-%d')

    # Auto-save for persistence
    salary_payments.to_csv("salary_payments.csv", index=False)

    print(f"✔ Salary marked PAID for {emp_id} for {month}/{year}")

    # Auto show updated report
    get_paid_status(month, year)


In [ ]:
mark_salary_as_paid("E001", 12, 2025)


✔ Salary marked PAID for E001 for 12/2025

=== PAID Employees for 12/2025 ===
employee_id  net_salary  paid_date
       E001    24038.46 2025-12-06

=== PENDING Employees for 12/2025 ===
employee_id  net_salary
       E002    28846.15
       E003    24923.08
       E004    24000.00
       E005    28000.00
       E006    22153.85
       E007    22000.00
       E008    23000.00
       E009    25653.85
       E010    29807.69
       E011    20192.31
       E012    25961.54
       E013    23000.00
       E014    21153.85
       E015    28000.00
       E016    25000.00
       E017    26769.23
       E018    27692.31
       E019    21230.77
       E020    25961.54
       E021    25000.00
       E022    22115.38
       E023    28000.00
       E024    28615.38
       E025    25653.85
       E026    22115.38
       E027    21153.85
       E028    24000.00
       E029    30000.00
       E030    23884.62
       E031    23692.31
       E032    23000.00
       E033    22153.85
       E034    24538.

In [ ]:
mark_salary_as_paid("E040", 12, 2025)


✔ Salary marked PAID for E040 for 12/2025

=== PAID Employees for 12/2025 ===
employee_id  net_salary  paid_date
       E001    24038.46 2025-12-06
       E040    25000.00 2025-12-06

=== PENDING Employees for 12/2025 ===
employee_id  net_salary
       E002    28846.15
       E003    24923.08
       E004    24000.00
       E005    28000.00
       E006    22153.85
       E007    22000.00
       E008    23000.00
       E009    25653.85
       E010    29807.69
       E011    20192.31
       E012    25961.54
       E013    23000.00
       E014    21153.85
       E015    28000.00
       E016    25000.00
       E017    26769.23
       E018    27692.31
       E019    21230.77
       E020    25961.54
       E021    25000.00
       E022    22115.38
       E023    28000.00
       E024    28615.38
       E025    25653.85
       E026    22115.38
       E027    21153.85
       E028    24000.00
       E029    30000.00
       E030    23884.62
       E031    23692.31
       E032    23000.00
       E03